In [ ]:
# https://wikidocs.net/58686

---
# 02. nn.Module로 구현하는 로지스틱 회귀

In [1]:
# nn.linear() 의 결과를 nn.sigmoid() 거치게 하여 로지스틱 회귀 가설을 만들자

In [3]:
# 라이브러리
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
# 랜덤시드 고정
torch.manual_seed(1)

In [5]:
# x_train, y_train 데이터를 텐서로 선언
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [6]:
# nn.Sequential() 을 이용해서 nn.linear와 nn.sigmoid를 이어주자
model = nn.Sequential(
    nn.Linear(2, 1),    # input_dim = 2, output_dim = 1
    nn.Sigmoid()        # 출력을 시그모이드 함수를 거친다
)

In [7]:
# 현재 w, b는 초기화 된 상태. 훈련 데이터를 넣어 예측값을 확인하자
model(x_train)

tensor([[0.4020],
        [0.4147],
        [0.6556],
        [0.5948],
        [0.6788],
        [0.8061]], grad_fn=<SigmoidBackward>)

In [8]:
# y_train과 같은 6*1 형태의 텐서가 나온다.
# 이제 경사 하강법을 사용하여 훈련해보자

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번 마다 로그 출력
    if epoch % 20 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])  # 0.5가 넘을 때만 True로
        correct_prediction = prediction.float() == y_train   # 예측값이 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction)        # 정확도를 계산
        print('Epoch : {:4d}/{}   Cost : {:.6f}   Accuracy : {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy*100
        ))

Epoch :    0/1000   Cost : 0.539713   Accuracy : 83.33%
Epoch :   20/1000   Cost : 0.441875   Accuracy : 66.67%
Epoch :   40/1000   Cost : 0.316358   Accuracy : 83.33%
Epoch :   60/1000   Cost : 0.220498   Accuracy : 100.00%
Epoch :   80/1000   Cost : 0.157299   Accuracy : 100.00%
Epoch :  100/1000   Cost : 0.134272   Accuracy : 100.00%
Epoch :  120/1000   Cost : 0.118297   Accuracy : 100.00%
Epoch :  140/1000   Cost : 0.105779   Accuracy : 100.00%
Epoch :  160/1000   Cost : 0.095704   Accuracy : 100.00%
Epoch :  180/1000   Cost : 0.087420   Accuracy : 100.00%
Epoch :  200/1000   Cost : 0.080486   Accuracy : 100.00%
Epoch :  220/1000   Cost : 0.074595   Accuracy : 100.00%
Epoch :  240/1000   Cost : 0.069526   Accuracy : 100.00%
Epoch :  260/1000   Cost : 0.065118   Accuracy : 100.00%
Epoch :  280/1000   Cost : 0.061247   Accuracy : 100.00%
Epoch :  300/1000   Cost : 0.057820   Accuracy : 100.00%
Epoch :  320/1000   Cost : 0.054764   Accuracy : 100.00%
Epoch :  340/1000   Cost : 0.05202

In [9]:
# 기존의 데이터를 가지고 예측해보자
model(x_train)

tensor([[2.7616e-04],
        [3.1595e-02],
        [3.8959e-02],
        [9.5624e-01],
        [9.9823e-01],
        [9.9969e-01]], grad_fn=<SigmoidBackward>)

In [11]:
# 훈련 후의 w ,b 를 확인해보자
print(list(model.parameters()))

[Parameter containing:
tensor([[3.2534, 1.5181]], requires_grad=True), Parameter containing:
tensor([-14.4839], requires_grad=True)]


In [12]:
# 결과적으로
# 위의 인공 신경망은 다음과 같은 다중 로지스틱 회귀를 표현한다

# H(x) = sigmoid(x1w1 + x2w2 + b)